# Converting the Spotify Dataset
This Jupyter Notebook will convert the dataset in `dataset/songs_normalize.csv` to and updated version of the dataset. The main updates are below.

- Append a new column for each genre in the original `genres` column. The value of each column is either True if that song is part of that genre, or False otherwise.
- Append a new column called `hasFeature` based on whether the song has a feature on the song title. A song is classified as having a "feature" if the title has one of the following keywords: `'feature', 'feat', 'ft', 'featuring', '(with', 'vs', 'vs.'`.

The updated dataset is stored in `dataset/songs_updated_v4.csv`.

In [1]:
import pandas as pd

In [2]:
# Load dataset
song_df = pd.read_csv('../dataset/songs_normalize.csv')


## Adding the `isGenre` columns
To create a new column for each genre, first we must find all the possible genres based on the values in the dataset. The original genre column contains a comma-separated list of genres. This will create a `set` with all the values of genre that we can find.

Then we create a dictionary with the key as the genre name and the values as a list of boolean values based on whether the song is part of the genre or not.

There are a few caveats to note:
- There are a few data points with no genre. Their values are `set()`. Instead of including that column, we remove that column.
- We prefix the new column names with `is`. As an example, `ispop` is one of the columns.

In [3]:
# Get list of all genres
genre_set = set()
for genres in song_df['genre'].values:
    genre_list = genres.split(',')
    for genre in genre_list:
        genre_set.add(genre.strip())
print(genre_set)

{'blues', 'set()', 'World/Traditional', 'pop', 'metal', 'jazz', 'Dance/Electronic', 'latin', 'country', 'R&B', 'Folk/Acoustic', 'easy listening', 'hip hop', 'classical', 'rock'}


In [4]:
# Populate a dictionary with the genres as keys and a list of True/False as values
# depending on whether the song is of that genre or not
# Make sure to ignore 'set()'
genre_dict = dict()
for genre in genre_set:
    if genre == 'set()':
        continue
    is_genre = []
    for genres in song_df['genre'].values:
        if genre in genres:
            is_genre.append(True)
        else:
            is_genre.append(False)
    genre_cleaned = 'is_' + genre.lower().replace(' ', '_').replace('/', '_')
    print(f'{genre} -> {genre_cleaned}')
    genre_dict[genre_cleaned] = is_genre

blues -> is_blues
World/Traditional -> is_world_traditional
pop -> is_pop
metal -> is_metal
jazz -> is_jazz
Dance/Electronic -> is_dance_electronic
latin -> is_latin
country -> is_country
R&B -> is_r&b
Folk/Acoustic -> is_folk_acoustic
easy listening -> is_easy_listening
hip hop -> is_hip_hop
classical -> is_classical
rock -> is_rock


In [5]:
# Check output of genre_dict and convert to a data frame
for genre, is_genre in genre_dict.items():
    print(f'{genre}: {is_genre[:5]}')

genres_df = pd.DataFrame(genre_dict)
genres_df.head()

is_blues: [False, False, False, False, False]
is_world_traditional: [False, False, False, False, False]
is_pop: [True, True, True, False, True]
is_metal: [False, False, False, True, False]
is_jazz: [False, False, False, False, False]
is_dance_electronic: [False, False, False, False, False]
is_latin: [False, False, False, False, False]
is_country: [False, False, True, False, False]
is_r&b: [False, False, False, False, False]
is_folk_acoustic: [False, False, False, False, False]
is_easy_listening: [False, False, False, False, False]
is_hip_hop: [False, False, False, False, False]
is_classical: [False, False, False, False, False]
is_rock: [False, True, False, True, False]


,is_blues,is_world_traditional,is_pop,is_metal,is_jazz,is_dance_electronic,is_latin,is_country,is_r&b,is_folk_acoustic,is_easy_listening,is_hip_hop,is_classical,is_rock
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False,True
2,False,False,True,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,True
4,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [6]:
# Append the data frame to the existing one
song_updated_df = song_df.copy()
song_updated_df = pd.concat([song_updated_df, genres_df], axis=1, sort=True)
song_updated_df.head()


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,...,is_jazz,is_dance_electronic,is_latin,is_country,is_r&b,is_folk_acoustic,is_easy_listening,is_hip_hop,is_classical,is_rock
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,False,False,False,False,False,False,False,False,False,False
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,...,False,False,False,False,False,False,False,False,False,True
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,...,False,False,False,True,False,False,False,False,False,False
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,...,False,False,False,False,False,False,False,False,False,True
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,...,False,False,False,False,False,False,False,False,False,False


## Adding the `hasFeature` column
Now, time to search for the list of keywords in a song title to determine whether a song has a feature or not. Once we know this, we can append this new column as well to the dataset.

In [7]:
# Add a hasFeature column
# Cannot use just 'with' as this returns all song titles with 'with' in the actual
# song title. So instead use '(with' which seems to be more accurate
feature_keywords = {'feature', 'feat', 'ft', 'featuring', '(with', 'vs', 'vs.'}

has_feature = []
for song_title in song_df['song']:
    has_feature.append(any(keyword in song_title for keyword in feature_keywords))

In [8]:
# Append new data to existing data frame
song_updated_df['has_feature'] = has_feature

## Remove duplicate song titles
There are some songs that appear in the dataset multiple times because they appear in the top hits of the year playlist on spotify throughout multiple years. This may skew our visualization and training data, so instead I will remove duplicates and add a new column that counts how many times a song has appeared on the top hits playlist.

In [9]:
song_count = dict()
for index, row in song_df.iterrows():
    key = row['song'] + ' - ' + row['artist']
    song_count[key] = song_count.get(key, 0) + 1

In [10]:
dp_rows = song_updated_df.duplicated(subset=['song', 'artist'], keep='first')
dp_rows = dp_rows[dp_rows == True]
len(dp_rows)

74

In [11]:
song_updated_df.drop_duplicates(subset=['song', 'artist'], keep='first', inplace=True)

In [12]:
song_count_list = []
for index, row in song_updated_df.iterrows():
    key = row['song'] + ' - ' + row['artist']
    song_count_list.append(song_count[key])

song_updated_df['num_years'] = song_count_list

In [13]:
song_updated_df[song_updated_df['num_years'] > 1].head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,...,is_latin,is_country,is_r&b,is_folk_acoustic,is_easy_listening,is_hip_hop,is_classical,is_rock,has_feature,num_years
20,Linkin Park,In the End,216880,False,2000,83,0.556,0.864,3,-5.870,...,False,False,False,False,False,False,False,True,False,2
36,Kylie Minogue,Spinning Around,207866,False,2000,55,0.761,0.662,6,-7.645,...,False,False,False,False,False,False,False,False,False,2
63,Craig David,Fill Me In,257200,False,2000,60,0.682,0.744,8,-6.981,...,False,False,True,False,False,True,False,False,False,2
85,Destiny's Child,"Independent Women, Pt. 1",221133,False,2001,65,0.730,0.602,6,-3.782,...,False,False,True,False,False,False,False,False,False,2
90,Gabrielle,Rise,219093,False,2001,60,0.558,0.481,8,-9.487,...,False,False,True,False,False,False,False,False,False,2


## Saving the new dataset
The dataset has been updated as we had hoped. Now we can save the new dataset under a new name `songs_updated_v4.csv`!

In [14]:
# Write updated data frame to CSV
song_updated_df.to_csv('../dataset/songs_updated_v4.csv', index=False)